In [1]:
from procDataSet import TrainingQuery
from bert_serving.client import BertClient
from multiprocessing import Pool
from torch.utils.data import TensorDataset, DataLoader, Dataset

import torch
import torch.nn as nn
import torch.optim  as optim
import argparse
from keras.utils import to_categorical
import pandas as pd
import numpy as np
from torch.functional import F

import os
import jieba
from sklearn.metrics.pairwise import cosine_similarity
from IPython.display import clear_output

import jieba.posseg as pseg

jieba.load_userdict(os.path.join('data', 'dict.txt.big'))
[jieba.add_word(i, freq=None, tag=None) for i in ['不支持','文林苑', '都更案','十八趴','證所稅','前瞻建設']]

Using TensorFlow backend.
Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.276 seconds.
Prefix dict has been built succesfully.


[None, None, None, None, None, None]

In [2]:
top_num = 300

In [3]:
with open('data/stop_word_zhtw.txt') as file:
    data = file.read()
    
stop_words = data.split('\n')
stop_words += ['「', '」', '，', '\n', '）', '（', ')', '(']

In [4]:
from sklearn.model_selection import train_test_split
folder = 'data/'
raw_training_data = pd.read_csv(os.path.join(folder,'TD.csv'))
news_urls = pd.read_csv(os.path.join(folder,'NC_1.csv'))
contents = pd.read_json(os.path.join(folder,'url2content.json'), typ=pd.Series)

## sort the contents by index
keys, content_list = contents.keys(), contents.values

In [5]:
test_query = np.array(pd.read_csv('./data/QS_1.csv').Query)

In [6]:
for test_id , text_q in enumerate(test_query):
    print(jieba.lcut(text_q))

['通姦', '在', '刑法', '上', '應該', '除罪化']
['應該', '取消', '機車', '強制', '二段式', '左轉', '(', '待轉', ')']
['支持', '博弈', '特區', '在', '台灣', '合法化']
['中華', '航空', '空服員', '罷工', '是', '合理', '的']
['性交易', '應該', '合法化']
['ECFA', '早', '收清單', '可', '（', '有', '）', '達到', '其', '預期', '成效']
['應該', '減免', '證所稅']
['贊成', '中油', '在', '觀塘', '興建', '第三', '天然氣', '接收站']
['支持', '中國', '學生', '納入', '健保']
['支持', '臺灣', '中小學', '（', '含', '高職', '、', '專科', '）', '服儀', '規定', '（', '含髮', '、', '襪', '、', '鞋', '）', '給予', '學生', '自主']
['不支持', '使用', '加密', '貨幣']
['不支持', '學雜費', '調漲']
['同意', '政府', '舉債', '發展', '前瞻建設', '計畫']
['支持', '電競', '列入', '體育競技']
['反對', '台鐵', '東移', '徵收案']
['支持', '陳前總統', '保外就醫']
['年金', '改革', '應', '取消', '或', '應', '調降', '軍公教', '月', '退之', '優存', '利率', '十八趴']
['同意', '動物', '實驗']
['油價', '應該', '凍漲', '或', '緩漲']
['反對', '旺旺', '中時', '併購', '中嘉']


In [7]:
import multiprocessing

NumberCPU = multiprocessing.cpu_count()
jieba.initialize()

def jbcut(x):
    if x is not None:
        sen = jieba.lcut(x, cut_all=False)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None

def psegcut(x):
    if x is not None:
        sen = pseg.lcut(x)
        sen = [i for i in sen if i not in stop_words]
        return sen
    else:
        return None
    
pool = multiprocessing.Pool(processes=NumberCPU)
sentence_arr = pool.map(jbcut,content_list)
pool.close()
pool.join()

In [8]:
# sentence_arr = np.load('arr.npy', allow_pickle=True)

## okapi model

In [9]:
from gensim.summarization import bm25
bm25Model = bm25.BM25(sentence_arr)

In [10]:
stands = ['支持', '應該', '反對', '贊成', '不'] 

In [52]:
train_query = list(set(raw_training_data.Query))

y_train = []
y_index = {}
for i  in train_query:
    index = np.where(raw_training_data.Query==i)
    data = raw_training_data.iloc[index]
    y = dict(zip(data.News_Index,data.Relevance))
    y_idx = [ (int(idx.split('_')[1])-1, rel )  for idx, rel in zip(data.News_Index, data.Relevance)]
    y_train.append(y)
    y_index[i] = [y_idx]

In [53]:
common_query = set(train_query) & set(test_query)
common_query = list(common_query)
common = [ train_query.index(i) for i in common_query]

In [54]:
[ len(list(y_train[i].values())) for i in common]

[139, 95, 40, 230, 230]

In [55]:
[ np.sum(list(y_train[i].values())) / 300 for i in range(20)]

[0.6933333333333334,
 2.7866666666666666,
 2.296666666666667,
 0.7066666666666667,
 0.23,
 1.4466666666666668,
 0.24333333333333335,
 1.3,
 0.14,
 1.0533333333333332,
 1.3733333333333333,
 0.5766666666666667,
 0.21333333333333335,
 0.2833333333333333,
 1.3233333333333333,
 1.35,
 2.1933333333333334,
 0.9433333333333334,
 0.77,
 2.6466666666666665]

In [149]:
import numpy as np

def histogram_equalization(image, number_bins=256):
    # from http://www.janeriksolem.net/2009/06/histogram-equalization-with-python-and.html

    # get image histogram
    image_histogram, bins = np.histogram(image.flatten(), number_bins, density=True)
    cdf = image_histogram.cumsum() # cumulative distribution function
    cdf = 100 * cdf / cdf[-1] # normalize

    # use linear interpolation of cdf to find new pixel values
    image_equalized = np.interp(image.flatten(), bins[:-1], cdf)
    return image_equalized

In [150]:
histogram_equalization(scores)

array([78.19874842, 19.22857733, 50.37420267, ..., 92.65986579,
       84.33728214, 66.55927755])

In [171]:
total_scores = list()
for test_id , text_q in enumerate(train_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    
    scores = np.array(bm25Model.get_scores(text))
    scores = histogram_equalization(scores)
    
    top_query = np.argsort(scores)[::-1][:5]
    irrelevant_query = np.argsort(scores)[:5]
    
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:200]
        top_words = [x[0] for x in top_words]
        text += top_words
        
    expansion_score = np.array(bm25Model.get_scores(text))
    expansion_score = histogram_equalization(expansion_score)
    scores += 5 * expansion_score
    
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    
    validation_score = 0
    for a in ans:
        if a in y_train[test_id].keys():
            validation_score += y_train[test_id][a]
            
    total_scores.append(validation_score/top_num)
    print("{} / {} ".format(validation_score,(np.sum(list(y_train[test_id].values())))))

0 反對旺旺中時併購中嘉 ['反對', '旺旺', '中時', '併購', '中嘉']
top query num 7
152 / 208 
1 國際賽事會場內應該可以持中華民國國旗 ['國際賽', '事', '會場', '應該', '持', '中華民國', '國旗']
top query num 7
132 / 836 
2 贊同課綱微調 ['贊同', '課綱', '微調']
top query num 7
210 / 689 
3 支持正名「臺灣」參與國際運動賽事 ['支持', '正名', '臺灣', '參與', '國際', '運動', '賽事']
top query num 7
143 / 212 
4 油價應該凍漲或緩漲 ['油價', '應該', '凍漲', '緩漲']
top query num 7
43 / 69 
5 贊成流浪動物零撲殺 ['贊成', '流浪', '動物', '零', '撲殺']
top query num 7
294 / 434 
6 拒絕公投通過門檻下修 ['拒絕', '公投', '門檻', '下修']
top query num 7
32 / 73 
7 反對二代健保規定 ['反對', '二代', '健保', '規定']
top query num 7
238 / 390 
8 同意動物實驗 ['同意', '動物', '實驗']
top query num 7
42 / 42 
9 核四應該啟用 ['核四', '應該', '啟用']
top query num 7
132 / 316 
10 臺灣應開放含瘦肉精(萊克多巴胺)之美國牛肉進口 ['臺灣', '應', '開放', '含', '瘦肉精', '萊克', '多巴胺', '美國', '牛肉', '進口']
top query num 7
92 / 412 
11 十二年國教高中職「免學費補助」適用對象增加是不對的 ['十二年', '國教', '高中職', '免', '學費', '補助', '適用', '對象', '增加', '不對']
top query num 7
125 / 173 
12 反對無圍牆校園 ['反對', '圍牆', '校園']
top query num 7
59 / 64 
13 贊成文林苑都更案可依法拆除王家 ['贊成', '文林苑', '都更案', '

In [172]:
np.mean(total_scores)

0.4963333333333334

In [166]:
total_scores = list()
for test_id , text_q in enumerate(test_query):
    text = jieba.lcut(text_q)
    text = [ t for t in text if t not in stop_words]
    
    scores = np.array(bm25Model.get_scores(text))
    scores = histogram_equalization(scores)
    
    top_query = np.argsort(scores)[::-1][:5]
    irrelevant_query = np.argsort(scores)[:5]
    
    print(test_id ,text_q, text)
    print("top query num {}".format(len(top_query)))
    
    for query in top_query:
        all_words = [ (sentence_arr[query][cnt], bm25Model.get_score(sentence_arr[query], cnt))
                     for cnt, i in enumerate(sentence_arr[query])]

        all_words = sorted(all_words,key=lambda x:(x[1]))[::-1]
        top_words = all_words[:200]
        top_words = [x[0] for x in top_words]
        text += top_words
    
    expansion_score = np.array(bm25Model.get_scores(text))
    expansion_score = histogram_equalization(expansion_score)
    scores += 6 * expansion_score
    
    top_num = 300
    keys = pd.DataFrame(np.argsort(scores)[::-1][:top_num])
    ans = keys[0].apply(lambda x: 'news_{:06d}'.format(x+1))
    
    delta = np.max(scores) 
    if text_q in y_index.keys():
        for idx, rel in (y_index[text_q][0]):
            scores[idx] += delta * rel
    
    total_scores.append(scores)

0 通姦在刑法上應該除罪化 ['通姦', '刑法', '應該', '除罪化']
top query num 5
1 應該取消機車強制二段式左轉(待轉) ['應該', '取消', '機車', '強制', '二段式', '左轉', '待轉']
top query num 5
2 支持博弈特區在台灣合法化 ['支持', '博弈', '特區', '台灣', '合法化']
top query num 5
3 中華航空空服員罷工是合理的 ['中華', '航空', '空服員', '罷工', '合理']
top query num 5
4 性交易應該合法化 ['性交易', '應該', '合法化']
top query num 5
5 ECFA早收清單可（有）達到其預期成效 ['ECFA', '早', '收清單', '達到', '預期', '成效']
top query num 5
6 應該減免證所稅 ['應該', '減免', '證所稅']
top query num 5
7 贊成中油在觀塘興建第三天然氣接收站 ['贊成', '中油', '觀塘', '興建', '第三', '天然氣', '接收站']
top query num 5
8 支持中國學生納入健保 ['支持', '中國', '學生', '納入', '健保']
top query num 5
9 支持臺灣中小學（含高職、專科）服儀規定（含髮、襪、鞋）給予學生自主 ['支持', '臺灣', '中小學', '含', '高職', '專科', '服儀', '規定', '含髮', '襪', '鞋', '給予', '學生', '自主']
top query num 5


KeyboardInterrupt: 

## Test the model

In [162]:
top_num = 300
search_result = np.zeros((20,top_num))
for cnt,i in enumerate(total_scores):
    keys = np.argsort(i)[::-1][:top_num]
    search_result[cnt] += keys
    
search_result = search_result.astype(np.int)

df = pd.DataFrame()
df['Query_Index'] = ['q_{:02d}'.format(i+1) for i in range(20)]

for i in range(top_num):
    df['Rank_{:03d}'.format(i+1)] = search_result[:, i] + 1

for i, row in df.iterrows():
    df.iloc[i, 1:] = df.iloc[i, 1:].apply(lambda x: 'news_{:06d}'.format(x))

In [164]:
fname = 'simple35.csv'
assert fname not in (os.listdir('output/'))

In [165]:
df.to_csv('output/' + fname, index=False)

# search_result[0]